In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [41]:
# Remove metadata and header rows
file_path = 'data/cumulative_2024.03.14_10.24.31.csv'

data_rows_clean = []

with open(file_path, mode='r', newline='') as file:
    reader = csv.reader(file)
    # Skip the first 53 lines (headers and metadata)
    for _ in range(53):
        next(reader)
    # Now read the actual data
    data_rows_clean = [row for row in reader]

# Convert to dataframe
df_cumulative = pd.DataFrame(data_rows_clean[1:], columns=data_rows_clean[0])

# Convert koi_disposition false positive to 0, candidate to 1, confirmed to 2
df_cumulative['koi_disposition'] = df_cumulative['koi_disposition'].map({'FALSE POSITIVE': 0, 'CANDIDATE': 1, 'CONFIRMED': 2})

# Convert koi_pdisposition false positive to 0, candidate to 1
df_cumulative['koi_pdisposition'] = df_cumulative['koi_pdisposition'].map({'FALSE POSITIVE': 0, 'CANDIDATE': 1})

# Convert columns to float
non_numeric_columns = ['kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_pdisposition', 'koi_tce_delivname']
numeric_columns = [col for col in df_cumulative.columns if col not in non_numeric_columns]

for column in numeric_columns:
    df_cumulative[column] = pd.to_numeric(df_cumulative[column], errors='coerce')

# Remove empty columns
df_cumulative = df_cumulative.drop(['koi_teq_err1', 'koi_teq_err2'], axis='columns')

# Remove rows that have empty koi_impact or koi_duration values
df_cumulative = df_cumulative.dropna(subset=['koi_impact', 'koi_duration'])
display(df_cumulative)


,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,2,1,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,2,1,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,,1,1,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,,0,0,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,2,1,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,10090151,K07985.01,,0,0,0.000,0,1,1,0,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9560,10128825,K07986.01,,1,1,0.497,0,0,0,0,...,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9561,10147276,K07987.01,,0,0,0.021,0,0,1,0,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385
9562,10155286,K07988.01,,1,1,0.092,0,0,0,0,...,-128.0,2.992,0.030,-0.027,7.824,0.223,-1.896,296.76288,47.145142,10.998


In [27]:
# convert to csv
df_cumulative.to_csv('data/cumulative_clean.csv', index=False)

In [4]:
# Scrape data columns from the Kepler Objects of Interest html.

# Read in the html file and soup it 
with open('data/Data columns in Kepler Objects of Interest Table.html','r') as f:
    html = f.read()
    soup = BeautifulSoup(html, 'html.parser')

# "Database Column Names" Colummn
column_name = []
for element in soup.find_all('td', class_='name'):
    column_name.append(element.text.strip())

# "Table Label" Column
tabel_labels = []
for element in soup.find_all('td', class_='label'):
    text = element.text.strip()
    if text != '' and '_str' not in text:
        tabel_labels.append(element.text.strip())

# "Description" Column
description = []
for element in soup.find_all('td', class_='description'):
    description.append(element.text.strip())

# "Uncertainty" Column  
uncertainty = []
columns = soup.find_all('tr', class_='column')

for item in columns:
    uncertainty_elements = item.find_all('td', class_='uncertainty')
    if not uncertainty_elements: 
        uncertainty.append(None)
    else:
        for element in uncertainty_elements:
            element = element.text.strip()
            if element!= '&nbsp;' and element != '':
                uncertainty.append(element)
            else:
                uncertainty.append(None)

# Create a dataframe
data_dict = {'Database Column Name': column_name, 
             'Table Label': tabel_labels, 
             'Description': description, 
             'Uncertainties Column (positive +) (negative - )': uncertainty}

df_columns = pd.DataFrame(data_dict)
df_columns.replace('[†]','', regex=True,inplace=True)

relavent_columns = df_columns[df_columns['Database Column Name'].isin(df_cumulative.columns)]
display(relavent_columns)

,Database Column Name,Table Label,Description,Uncertainties Column (positive +) (negative - )
0,kepid,Kepler Identification or KepID,"Target identification number, as listed in the...",None
1,kepoi_name,KOI Name,A number used to identify and track a Kepler O...,None
2,kepler_name,Kepler Name,"Kepler number name in the form ""Kepler-N,"" plu...",None
3,koi_disposition,Exoplanet Archive Disposition,The category of this KOI from the Exoplanet Ar...,None
6,koi_pdisposition,Disposition Using Kepler Data,The pipeline flag that designates the most pro...,None
7,koi_score,Disposition Score,A value between 0 and 1 that indicates the con...,None
8,koi_fpflag_nt,Not Transit-Like Flag,A KOI whose light curve is not consistent with...,None
9,koi_fpflag_ss,Stellar Eclipse Flag,A KOI that is observed to have a significant s...,None
10,koi_fpflag_co,Centroid Offset Flag,The source of the signal is from a nearby star...,None
11,koi_fpflag_ec,Ephemeris Match Indicates Contamination Flag,The KOI shares the same period and epoch as an...,None


In [5]:
# Convert to csv file
relavent_columns.to_csv('data/Data columns in Kepler Objects of Interest Table.csv', index=False)

This research has made use of the NASA Exoplanet Archive, which is operated by the California Institute of Technology, under contract with the National Aeronautics and Space Administration under the Exoplanet Exploration Program.

DOI 10.26133/NEA4